<a href="https://colab.research.google.com/github/qharo/googleColab/blob/master/recurrentNeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x
from tensorflow.keras.preprocessing import sequence
from tensorflow import keras
import tensorflow as tf
import os
import numpy as np

#GET THE FILE
path_to_file = keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

#ENCODING
vocab = sorted(set(text))
char2idx = {u:i for i,u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
    return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

#DECODING
def int_to_text(ints):
    try:
        ints = ints.numpy()
    except:
        pass
    return ' '.join(idx2char[ints])

#MOLDING THE TRAINING EXAMPLES
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

#MAKING INPUT/OUTPUT DATA
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

#MAKE TRAINING BATCHES
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)
EMBEDDING_DIM = 256
RNN_UNITS = 1024

BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder = True)

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
        keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer="glorot_uniform"),
        keras.layers.Dense(vocab_size),
    ])
    return model

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)

#CREATING A LOSS FUNCTION
def loss(labels, logits):
    return keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

#COMPILE MODEL
model.compile(optimizer='adam', loss=loss)

#CHECKPOINTS
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix, 
    save_weights_only=True
)

#TRAINING MODEL
history = model.fit(data, epochs=100, callbacks=[checkpoint_callback])

#LOADING MODEL
model = build_model(VOCAB_SIZE, EMBEDDING_DIM,RNN_UNITS, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1,None]))
    

Epoch 1/100
172/172 [==============================] - 9s 54ms/step - loss: 2.5955
Epoch 2/100
172/172 [==============================] - 9s 54ms/step - loss: 1.9013
Epoch 3/100
172/172 [==============================] - 9s 54ms/step - loss: 1.6460
Epoch 4/100
172/172 [==============================] - 9s 54ms/step - loss: 1.5087
Epoch 5/100
172/172 [==============================] - 9s 54ms/step - loss: 1.4273
Epoch 6/100
172/172 [==============================] - 9s 54ms/step - loss: 1.3700
Epoch 7/100
172/172 [==============================] - 9s 54ms/step - loss: 1.3241
Epoch 8/100
172/172 [==============================] - 9s 54ms/step - loss: 1.2858
Epoch 9/100
172/172 [==============================] - 9s 54ms/step - loss: 1.2491
Epoch 10/100
172/172 [==============================] - 9s 54ms/step - loss: 1.2143
Epoch 11/100
172/172 [==============================] - 9s 54ms/step - loss: 1.1793
Epoch 12/100
172/172 [==============================] - 9s 54ms/step - loss: 1.1432
E

In [ ]:
#ACTUALLY GENERATING TEXT
def generate_text(model, start_string):
    num_generate = 800
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []
    temmperature = 1.0
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions/temmperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])
    return (start_string + ' '.join(text_generated))

inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: good eve
good even t . 
 
 G L O U C E S T E R : 
 S a y ,   d o   n o t   s w e a r ,   m y   l o r d   o f   B u c k i n g h a m . 
 
 C A T E S B E R : 
 O   m y   m i s e r y ,   P e w i s   a n d   t i l l   t h e i r   p o w e r 
 O f   g u i l t y   o f   t h e   w e l k i n g   s t a d d s ,   t y b a l t   f r o m   m e ! 
 W h o   t o l d   m e ,   g e n t l e   N o r f o l k ,   t h u s   e x c h a n g e , 
 O f   t h e   m o s t   s e e d s   w i t h   h e r ,   l e t   h e r   b e   w h a t   l o v e   i n   d e a t h ! 
 
 C l o w n : 
 A l a c k ,   p o o r   M a r g a r e t ,   y o u   b u y   d i e   t h e r e ,   m y   f r i e n d ? 
 I s   n e i t h e r   h e a d ,   b e i n g   n e ' e r   s i t h 
 T h a t   w a n t   n o   e a r s   a b r o a d - - 
 T o   p l a g u e   t h e e   f o r   t h e   f i e l d   t o   g o ' n . 
 
 K I N G   H E N R Y   V I : 
 S i r   W i l l i a m   B r a n d o n ,   a n d   y o u   b o t h   s h e   s i n g s